In the second phase, we will be doing machine learning to estimate porosity
We read our three batches as follow

First step, reading data

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
os.environ["TF+CPP_MIN_LOG_LEVEL"]="2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
fd = '../input/batch-combined/Porosity Image J.xlsx'

# tabular data for first and second batches are placed in first and second sheets of our excel sheet, respectively

batch_train = pd.DataFrame(pd.read_excel(fd, sheet_name = 0))
batch_test = pd.DataFrame(pd.read_excel(fd, sheet_name = 'Batch3'))
batch_train['ID'] = batch_train['ID'].str.replace('batch1:', 'batch1_')
batch_train['ID'] = batch_train['ID'].str.replace('batch2:', 'batch2_')
batch_test['ID'] = batch_test['ID'].str.replace('batch3:', '')
batch_test.head(3)

Second step, convolutional neural network to classify images

Using convolutional neural network (cnn), we classify images into four different categories
0% to 10% is class one
10% to 20% is class two
20% to 30% is class three
30% to 100% is class four

In [ ]:
class_of_porosity=[]
for i in range(len(batch_train)):
    if batch_train['porosity(%)'][i] <= 10:
        class_of_porosity.append(0)
    elif batch_train['porosity(%)'][i]> 10 and batch_train['porosity(%)'][i] <= 20:
        class_of_porosity.append(1)
    elif batch_train['porosity(%)'][i]> 20 and batch_train['porosity(%)'][i] <= 30:
        class_of_porosity.append(2)
    else:
        class_of_porosity.append(3)
batch_train['category']=pd.DataFrame(class_of_porosity)
batch_train.head(4)

We do porosity classification for test dataset as well

In [ ]:
class_of_porosity=[]
for i in range(len(batch_test)):
    if batch_test['porosity(%)'][i] <= 10:
        class_of_porosity.append(0)
    elif batch_test['porosity(%)'][i]> 10 and batch_test['porosity(%)'][i] <= 20:
        class_of_porosity.append(1)
    elif batch_test['porosity(%)'][i]> 20 and batch_test['porosity(%)'][i] <= 30:
        class_of_porosity.append(2)
    else:
        class_of_porosity.append(3)
batch_test['category']=pd.DataFrame(class_of_porosity)
batch_test.head(4)

Then, we represent classified porosity for train(blue) and test(orange) data frames.

In [ ]:
batch_train.category.hist();
batch_test.category.hist();

Third step, application of cnn for image classification

Image IDs will be used to help us read images, also the categories will be the labels assigned to each figure

In [ ]:
# df=pd.read_excel(fd, sheet_name = 'Batch2'+'train.excel')
file_paths=batch_train['ID'].values
labels=batch_train['category'].values

In this step, we split file path from labels to define training dataset

In [ ]:
ds_train=tf.data.Dataset.from_tensor_slices((file_paths,labels))

In [ ]:
file_paths=batch_test['ID'].values
labels=batch_test['category'].values

We also define testing dataset as follow, which has two components (file path and label)

In [ ]:
ds_test=tf.data.Dataset.from_tensor_slices((file_paths,labels))

The data from first two batches are combined. Therefore, we have 200 data to train cnn.

In [ ]:
def read_image(image_file, label):
    image=tf.io.read_file('../input/batch-combined/batch1and2/'+ image_file+'.png')
    image=tf.image.decode_image(image,dtype=tf.float32)
    return image, label
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label 
ds_train=ds_train.map(read_image).map(augment).batch(2)

Similarly, testing data (100 rows of data) is imported to be used as validation dataset

In [ ]:
def read_image(image_file, label):
    image=tf.io.read_file('../input/cee-498-project8-pore-in-concrete/batch3/batch3/'+ image_file+'.png')
    image=tf.image.decode_image(image,dtype=tf.float32)
    return image, label

Batch input dimension is 2

In [ ]:
ds_test=ds_test.map(read_image).batch(2)

To avoid overfitting, images will analysis using different preprocessing methods including flip, rotation, resizing, etc.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomFlip("vertical"),
        layers.experimental.preprocessing.RandomRotation(0.5),
        layers.experimental.preprocessing.RandomHeight(0.5),
        layers.experimental.preprocessing.RandomWidth(0.5),
        layers.experimental.preprocessing.RandomCrop(100, 100, seed=None, name=None),
        layers.experimental.preprocessing.Resizing(100, 100, interpolation='bilinear', name=None),
#         layers.experimental.preprocessing.RandomContrast(1, seed=None, name=None)
        
    ]
)

In [ ]:
height, width = 192, 256
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(height, width, 3),
    include_top=False,
)

base_model.trainable = True

# Create new model on top
inputs = keras.Input(shape=(height, width, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

from tensorflow.keras.optimizers import RMSprop



# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(inputs)
norm_layer.set_weights([mean, var])


# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
#x = layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# Add a final sigmoid layer for classification
x = layers.Dense(1024, activation='relu')(x)
# outputs = keras.layers.Dense(1)(x)
# model = keras.Model(inputs, outputs)

num_outputs = 4 # This is the number of output variables we want, 196 in this case.
outputs = keras.layers.Dense(num_outputs, activation="softmax")(x) # Use activation=softmax for classification, and activation=None for regression.
model = keras.Model(inputs, outputs)


# Configure and compile the model
#model = Model(base_model.input, x)
# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(lr=0.0001),
#               metrics=['acc'])

model.summary()

cnn hyperparameters are defined to get optimal results

In [ ]:
learning_rate = 5.0e-5
# model = tf.keras.models.Sequential()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

epochs = 75
history = model.fit(ds_train, epochs=epochs, validation_data=ds_test)

The next cell represents loss(error) of image classification vs. Epochs, for both training and validation datasets.

In [ ]:
plt.xlabel("Epochs")
plt.xlabel("Loss")
plt.plot(history.history['loss'], color='b', label="Loss")
plt.plot(history.history['val_loss'], color='g', label="Accuracy")
plt.show()

The next cell represents accuracy of image classification vs. Epochs, for both training dataset.

In [ ]:
plt.xlabel("Epochs")
plt.xlabel("Loss")
plt.plot(history.history['sparse_categorical_accuracy'], color='b', label="Loss")
# plt.plot(history.history['val_sparse_categorical_accuracy'], color='b', label="Loss")
# plt.legend()
plt.show()

Conclusion
cnn categorized images with 45% accuracy which is desired.